# Julia Draw Something

In [1]:
using DataFrames
using TensorFlow
using Distributions
using Images

I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcublas.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:116] Couldn't open CUDA library libcudnn.so.5. LD_LIBRARY_PATH: /usr/local/cuda/lib64
I tensorflow/stream_executor/cuda/cuda_dnn.cc:3459] Unable to load cuDNN DSO
I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcufft.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcurand.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcublas.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcudnn.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcufft.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUD

In [2]:
data_set_folder = "/raided/datasets/sketches_png/png/"
list_of_imgs = open(data_set_folder * "filelist.txt")
img_paths = readlines(list_of_imgs)[1:400]
close(list_of_imgs)

In [3]:
train_paths = Array(String, 350)
count = 1
for i = 1:350
    train_paths[i] = img_paths[count]
    count += 1
    if count % 71 == 0
        count += 10
    end
end

test_paths = Array(String, 50)
count = 71
for i = 1:50
    test_paths[i] = img_paths[count]
    count += 1
    if count % 81 == 0
        count += 70
    end
end

In [4]:
files_in_folder = readdir(data_set_folder)
labels = filter(x->isdir(data_set_folder * x) == true, files_in_folder)

250-element Array{String,1}:
 "airplane"     
 "alarm clock"  
 "angel"        
 "ant"          
 "apple"        
 "arm"          
 "armchair"     
 "ashtray"      
 "axe"          
 "backpack"     
 "banana"       
 "barn"         
 "baseball bat" 
 ⋮              
 "umbrella"     
 "van"          
 "vase"         
 "violin"       
 "walkie talkie"
 "wheel"        
 "wheelbarrow"  
 "windmill"     
 "wine-bottle"  
 "wineglass"    
 "wrist-watch"  
 "zebra"        

In [5]:
x = TensorFlow.placeholder(Float32)
y_ = TensorFlow.placeholder(Float32)
W = get_variable("weights", [77841, 5], Float32)
b = get_variable("bias", [5], Float32)

TensorFlow.Variable(<Operation 'bias'>,<Operation 'bias/Assign'>)

In [6]:
y = nn.softmax(x*W + b)
cross_entropy = reduce_mean(-reduce_sum(y_ .* log(y), reduction_indices=[2]))
train_step = train.minimize(train.GradientDescentOptimizer(.00001), cross_entropy)

W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.


<Tensor NoOp:1 shape=unknown dtype=?>

In [7]:
type DataLoader
    cur_id::Int
    order::Vector{Int}
end

TrainLoader() = DataLoader(1, shuffle(1:350))
TestLoader() = DataLoader(1, shuffle(1:50))

TestLoader (generic function with 1 method)

In [8]:
function next_batch(loader::DataLoader, batch_size)
    x = zeros(Float32, batch_size, 77841)
    y = zeros(Float32, batch_size, 5)
    for i = 1:batch_size
        path = train_paths[loader.order[loader.cur_id]][1:end-1]
        println(path)
        sketch_name = split(path, "/")[1]
        img = load(data_set_folder * path)
        img = restrict(restrict(img))
        flatten_img = reshape(img, 77841)
        
        x[i, :] = flatten_img
        label = find((x -> x == sketch_name), labels)[1]
        y[i, label] = 1.0
        loader.cur_id += 1
        if loader.cur_id > 350
            loader.cur_id = 1
            break
        end
    end
    x, y
end

next_batch (generic function with 1 method)

In [9]:
function get_test(loader::DataLoader)
    x = zeros(Float32, 50, 77841)
    y = zeros(Float32, 50, 5)
    for i = 1:50
        path = test_paths[loader.order[loader.cur_id]][1:end-1]
        sketch_name = split(path, "/")[1]
        img = load(data_set_folder * path)
        img = restrict(restrict(img))
        flatten_img = reshape(img, 77841)
        
        x[i, :] = flatten_img
        label = find((x -> x == sketch_name), labels)[1]
        println("--------------------------")
        println(label)
        println(loader.cur_id)
        y[i, label] = 1.0
        loader.cur_id += 1
        if loader.cur_id > 50
            println("========================")
            println(sketch_name)
            break
        end
    end
    x, y
end

get_test (generic function with 1 method)

In [10]:
loader = TrainLoader()
sess = Session()
run(sess, initialize_all_variables())
for i in 1:35
    batch = next_batch(loader, 10)
    run(sess, train_step, Dict(x=>batch[1], y_=>batch[2]))
    println("DONE BATCH ", i)
end

angel/187.png


I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:910] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
I tensorflow/core/common_runtime/gpu/gpu_device.cc:885] Found device 0 with properties: 
name: GeForce GTX 1070
major: 6 minor: 1 memoryClockRate (GHz) 1.683
pciBusID 0000:01:00.0
Total memory: 7.92GiB
Free memory: 7.82GiB
I tensorflow/core/common_runtime/gpu/gpu_device.cc:906] DMA: 0 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:916] 0:   Y 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:975] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0)


ant/296.png
alarm clock/113.png
alarm clock/120.png
apple/385.png
alarm clock/122.png
airplane/3.png
angel/165.png
apple/342.png
apple/400.png
DONE BATCH 1
ant/241.png
alarm clock/142.png
apple/379.png
apple/365.png
ant/320.png
apple/380.png
alarm clock/157.png
ant/319.png
alarm clock/138.png
apple/384.png
DONE BATCH 2
alarm clock/125.png
airplane/53.png
ant/256.png
apple/345.png
airplane/39.png
ant/251.png
alarm clock/145.png
airplane/52.png
airplane/69.png
alarm clock/146.png
DONE BATCH 3
airplane/25.png
alarm clock/134.png
apple/373.png
ant/272.png
apple/398.png
airplane/31.png
airplane/6.png
ant/300.png
ant/245.png
apple/387.png
DONE BATCH 4
ant/312.png
angel/212.png
angel/228.png
airplane/71.png
apple/348.png
airplane/29.png
airplane/56.png
alarm clock/114.png
airplane/5.png
apple/340.png
DONE BATCH 5
angel/235.png
ant/249.png
airplane/7.png
alarm clock/91.png
airplane/1.png
angel/204.png
alarm clock/140.png
alarm clock/124.png
airplane/43.png
airplane/17.png
DONE BATCH 6
alarm cl

In [11]:
correct_prediction = indmax(y, 2) .== indmax(y_, 2)
accuracy=reduce_mean(cast(correct_prediction, Float64))
test_loader = TestLoader()
testx, testy = get_test(test_loader)

println(run(sess, indmax(y, 2), Dict(x=>testx, y_=>testy)))
println(run(sess, indmax(y_, 2), Dict(x=>testx, y_=>testy)))
println(run(sess, accuracy, Dict(x=>testx, y_=>testy)))

--------------------------
1
1
--------------------------
4
2
--------------------------
2
3
--------------------------
5
4
--------------------------
2
5
--------------------------
4
6
--------------------------
1
7
--------------------------
4
8
--------------------------
5
9
--------------------------
5
10
--------------------------
2
11
--------------------------
2
12
--------------------------
3
13
--------------------------
5
14
--------------------------
3
15
--------------------------
5
16
--------------------------
2
17
--------------------------
2
18
--------------------------
5
19
--------------------------
5
20
--------------------------
2
21
--------------------------
4
22
--------------------------
1
23
--------------------------
2
24
--------------------------
3
25
--------------------------
4
26
--------------------------
1
27
--------------------------
1
28
--------------------------
2
29
--------------------------
3
30
--------------------------
3
31
-----------------

In [12]:
function sketch_input(path, sketch_name)
    x = zeros(Float32, 1, 77841)
    y = zeros(Float32, 1, 10)
    img = load(path)
    img = restrict(restrict(img))
    flatten_img = reshape(img, 77841)
    x[1, :] = flatten_img
    
    label = find((x -> x == sketch_name), labels)[1]
    y[1, label] = 1.0
    
    x, y
end

sketch_input (generic function with 1 method)

In [13]:
sketch_path = "/raided/datasets/sketches_png/png/airplane/79.png"
sketch_label = "airplane"

test_img, test_label = sketch_input(sketch_path, sketch_label)
what_it_thinks = labels[run(sess, indmax(y,2), Dict(x=>test_img, y_=>test_label))[1] + 1]
println("Is it an " * what_it_thinks * "? Answer: ", (sketch_label == what_it_thinks), ".")

Is it an angel? Answer: false.
